In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
import joblib

c:\Users\dshar\OneDrive\Pictures\Desktop\RAPTOR-RAG\raptor\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import joblib
results = joblib.load('ram.joblib')
docs_texts = joblib.load('ram_docs.joblib')

In [3]:
# # Initialize all_texts with leaf_texts
leaf_texts = docs_texts
all_texts = leaf_texts.copy()

# Iterate through the results to extract summaries from each level and add them to all_texts
for level in sorted(results.keys()):
    # Extract summaries from the current level's DataFrame
    summaries = results[level][1]["summaries"].tolist()
    # Extend all_texts with the summaries from the current level
    all_texts.extend(summaries)

In [8]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(host='172.19.14.186',port=19530)

collection_name = "ram1"

In [9]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=768,
    metric_type="COSINE",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
)

In [10]:
embedings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004").embed_documents(all_texts)

In [11]:
from tqdm import tqdm

data = []

for i, line in enumerate(tqdm(all_texts, desc="Creating embeddings")):
    data.append({"id": i, "vector": embedings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 357/357 [00:00<00:00, 224526.39it/s]


{'insert_count': 357,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

In [10]:
question = "Who is sita?"

In [11]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        GoogleGenerativeAIEmbeddings(model="models/text-embedding-004").embed_query(question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=3,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

In [12]:
search_res

data: ['[{\'id\': 278, \'distance\': 0.6976392269134521, \'entity\': {\'text\': \'The text you provided is an excerpt from a fictional novel, likely a reimagining of the Ramayana. It details a dramatic battle scene where Sita, wife of Rama, finds herself defending against Lankan soldiers who are searching for her. \\n\\nHere\\\'s a breakdown of the scene:\\n\\n* **Setting:** The scene takes place in a temporary camp, likely in the forest, showing signs of a fierce struggle. \\n* **Characters:**\\n    * **Sita:** The protagonist of the scene, portrayed as a strong and resourceful warrior skilled in archery and knife throwing. She is deeply loyal to her husband, Rama, and her people.\\n    * **Jatayu:** A brave Malayaputra captain and a father figure to Sita, described as a "Naga" with a vulture-like face due to a deformity. He is captured and brutally tortured by the Lankans.\\n    * **Khara:** The main antagonist, a cruel Lankan captain leading the search for Sita. He enjoys inflicting

In [13]:
import json

In [14]:
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "The text you provided is an excerpt from a fictional novel, likely a reimagining of the Ramayana. It details a dramatic battle scene where Sita, wife of Rama, finds herself defending against Lankan soldiers who are searching for her. \n\nHere's a breakdown of the scene:\n\n* **Setting:** The scene takes place in a temporary camp, likely in the forest, showing signs of a fierce struggle. \n* **Characters:**\n    * **Sita:** The protagonist of the scene, portrayed as a strong and resourceful warrior skilled in archery and knife throwing. She is deeply loyal to her husband, Rama, and her people.\n    * **Jatayu:** A brave Malayaputra captain and a father figure to Sita, described as a \"Naga\" with a vulture-like face due to a deformity. He is captured and brutally tortured by the Lankans.\n    * **Khara:** The main antagonist, a cruel Lankan captain leading the search for Sita. He enjoys inflicting pain and uses Jatayu's life as leverage. \n    * **Kumbhakarna:** Raavan'

In [15]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [16]:
context

'The text you provided is an excerpt from a fictional novel, likely a reimagining of the Ramayana. It details a dramatic battle scene where Sita, wife of Rama, finds herself defending against Lankan soldiers who are searching for her. \n\nHere\'s a breakdown of the scene:\n\n* **Setting:** The scene takes place in a temporary camp, likely in the forest, showing signs of a fierce struggle. \n* **Characters:**\n    * **Sita:** The protagonist of the scene, portrayed as a strong and resourceful warrior skilled in archery and knife throwing. She is deeply loyal to her husband, Rama, and her people.\n    * **Jatayu:** A brave Malayaputra captain and a father figure to Sita, described as a "Naga" with a vulture-like face due to a deformity. He is captured and brutally tortured by the Lankans.\n    * **Khara:** The main antagonist, a cruel Lankan captain leading the search for Sita. He enjoys inflicting pain and uses Jatayu\'s life as leverage. \n    * **Kumbhakarna:** Raavan\'s brother, pres

In [56]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Pull the prompt from LangChain Hub
prompt = hub.pull("rlm/rag-prompt")

# Post-processing function
def format_docs(context):
    return context

# Initialize the language model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-001")

# Chain definition
rag_chain = (
    {"context": RunnablePassthrough() | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "Who is sita?"

# Invoke the chain with the context and question
response = rag_chain.invoke({"context": context, "question": question})

# Print the response
print(response)

Sita is the protagonist of the novel excerpt. She is a young girl who is dealing with the death of her mother, Sunaina, and has a mysterious past surrounding her birth and adoption. 

